In [1]:
import sys
import os
import json
import requests
import pandas as pd

sys.path.append(os.path.abspath('../scripts'))

from strava_export import get_access_token
from export_data import update_google_sheet
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# Path for stava activities save
DATA_PATH = f"{os.getenv('PROJECT_PATH')}/data"

# Get the access token
access_token = get_access_token()

In [ ]:
# Use the access token to fetch activities
activities_response = requests.get(
    url='https://www.strava.com/api/v3/athlete/activities',
    headers={'Authorization': f'Bearer {access_token}'}
).json()

# Save activities to json file
with open(f'{DATA_PATH}/downloads/strava_activities.json', 'w') as f:
    json.dump(activities_response, f, indent=4)
    
# Create a list of dictionaries with the required fields
activities = pd.DataFrame([
    {
        'id': activity['id'],
        'activity': activity['name'],
        'distance': activity['distance'],
        'moving_time': activity['moving_time'],
        'elapsed_time': activity['elapsed_time'],
        'date': pd.to_datetime(activity['start_date_local']).strftime('%Y-%m-%d'),
        'average_speed': activity['average_speed'],
        'max_speed': activity['max_speed'],
        'has_heartrate': activity['has_heartrate'],
        'average_heartrate': activity.get('average_heartrate', None),
        'max_heartrate': activity.get('max_heartrate', None)
    }
    for activity in activities_response
])

In [ ]:
# Iterate over activities and collect calories/suffer score
activity_info = []
for id in activities['id']:
    # Use the access token to fetch activities
    activity_response = requests.get(
        url=f'https://www.strava.com/api/v3/activities/{id}',
        headers={'Authorization': f'Bearer {access_token}'}
    )

    # Extract calorie and suffer score entries from json response
    activity_data = activity_response.json()
    activity_info.append([id, activity_data.get('calories'), activity_data.get('suffer_score')])

# Additional activity information
activity_info = pd.DataFrame(activity_info, columns=['id', 'calories', 'suffer_score'])

# Join with activities
new_activities = activities.merge(activity_info, on='id', how='left')

# Select relevant columns for export
activity_columns = [
    'activity', 'date', 'calories', 'distance', 
    'moving_time', 'elapsed_time', 'average_speed', 'max_speed', 
    'average_heartrate', 'max_heartrate', 'suffer_score'
]
new_activities = new_activities.loc[:, activity_columns]

In [ ]:
# Save updated activities dataframe (don't overwrite previous save)
new_activities.to_csv(f'{DATA_PATH}/strava_activities_new.csv')

# Export to google sheet
update_google_sheet(new_activities, 1)